Добро пожаловать в Jupyter Notebook, веб-интерфейс интерактивного интерпретатора языка Python.

Эта программа выполняет моделирование макромолекулы как цепи с фиксированными валентными углами. Программа, исходный код которой вводится здесь, исполняется на удаленном сервере, а результаты выполнения отображаются на этой странице.

На первом шаге мы подготовим интерпретатор для выполнения моделирования: импортируем нужные модули, определим функции и константы.

Установите курсор в ячейку с кодом ниже, щелкнув по ней, и нажмите кнопку `Run` на панели инструментов сверху, чтобы исполнить шаг.

In [ ]:
%matplotlib notebook

import math
from math import sin, cos, tan, asin, acos, atan2, pi
import random
from statistics import mean

import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt


# Число мономерных звеньев / сегментов полимерной цепи
SEGMENT_COUNT = 200

# Валентный угол
ANGLE_DEG = 109


def normalize(v):
    norm = np.linalg.norm(v)
    return v / norm

def pdf(r):
    scale = math.sqrt(3 / mean([x**2 for x in distances]))
    x = r * scale
    return math.sqrt(2/pi) * x**2 * math.exp(-x**2/2) * scale

LENGTH = 1
angle = math.radians(ANGLE_DEG)
i_vector = np.array([1, 0, 0])

assert angle > 0
assert angle < pi

r = LENGTH
theta = pi - angle

def calc():
    points = [np.array([0, 0, 0]), np.array([0.1, 0.1, math.sqrt(0.98)])]
    for index in range(SEGMENT_COUNT - 1):
        prev_point = points[index]
        curr_point = points[index + 1]
        prev_vector = curr_point - prev_point
        circle_center = curr_point + prev_vector * cos(theta)
        circle_radius = LENGTH * sin(theta)

        u = normalize(np.cross(prev_vector, i_vector))
        v = normalize(np.cross(prev_vector, u))
        t = random.uniform(0, 2 * pi)
        next_point = circle_center + circle_radius * cos(t) * u + circle_radius * sin(t) * v
        points.append(next_point)

    distance = np.linalg.norm(points[0] - points[-1])
    return points, distance

def draw():
    points, distance = calc()
    fig = plt.figure(figsize=[12, 7], dpi=80)
    ax = fig.add_subplot(111, projection='3d')

    for idx, point in enumerate(points):
        ax.scatter(*point, c='b', marker='o')
        if idx != 0:
            ax.plot([points[idx - 1][0], points[idx][0]],
                    [points[idx - 1][1], points[idx][1]],
                    zs=[points[idx - 1][2], points[idx][2]],
                    color='blue')

    ax.scatter(*points[0], c='r', marker='o')
    ax.scatter(*points[-1], c='r', marker='o')
    ax.plot([points[-1][0], points[0][0]],
            [points[-1][1], points[0][1]],
            zs=[points[-1][2], points[0][2]],
            dashes=(1, 3),
            color='red')

    min_x = min(a[0] for a in points)
    min_y = min(a[1] for a in points)
    min_z = min(a[2] for a in points)
    max_x = max(a[0] for a in points)
    max_y = max(a[1] for a in points)
    max_z = max(a[2] for a in points)
    avg_x = np.mean([a[0] for a in points])
    avg_y = np.mean([a[1] for a in points])
    avg_z = np.mean([a[2] for a in points])
    max_scale_length = max([max_x - min_x, max_y - min_y, max_z - min_z])
    ax.set_xlim3d([avg_x - 0.4 * max_scale_length, avg_x + 0.4 * max_scale_length])
    ax.set_ylim3d([avg_y - 0.4 * max_scale_length, avg_y + 0.4 * max_scale_length])
    ax.set_zlim3d([avg_z - 0.4 * max_scale_length, avg_z + 0.4 * max_scale_length])
    
    return distance
  
distances = []

Отлично! Теперь можно переходить непосредственно к моделированию — нажмите `Run`, установив курсор в ячейку ниже, и будет сгенерирована и отображена случайная макромолекула согласно созданной модели.

In [ ]:
distance = draw()
distances.append(distance)
print('R = {}'.format(distance))

Мы смоделировали одну макромолекулу. Полученное расстояние между концами цепи, характеризующее её гибкость, выведено под изображением.
Попробуйте ещё несколько раз выполнить последний шаг, чтобы сгенерировать несколько разных макромолекул.

После этого давайте посмотрим на распределение (гистограмму) расстояния между концами цепи — для этого запустите следующий шаг.

In [ ]:
plt.hist(distances, bins=10)

Если вы запускали моделирование не слишком много раз, скорее всего, получившаяся гистограмма выглядит не очень понятно: слишком велика роль случайности в полученном распределении. Давайте попробуем сгенерировать 1000 макромолекул и выведем получившееся распределение (это займет несколько десятков секунд):

In [ ]:
NUMBER_OF_SIMULATONS = 1000
NUMBER_OF_HISTOGRAM_BINS = 20

distances = []
for _ in range(NUMBER_OF_SIMULATONS):
    _, distance = calc()
    distances.append(distance)

fig, ax = plt.subplots()
n, bins, patches = ax.hist(distances, NUMBER_OF_HISTOGRAM_BINS)
theoretical_pdf = [pdf(b) * NUMBER_OF_SIMULATONS * (bins[1] - bins[0])
                   for b in range(int(bins[-1]) + 1)]
ax.plot(range(int(bins[-1]) + 1), theoretical_pdf, color='red', linestyle='dashed')
ax.set_xlabel('Среднеквадратичное расстояние между концами цепи')
ax.set_ylabel('Число макромолекул')
ax.set_title('Распределение макромолекул по размерам')
plt.show()

Совсем другое дело! Видно, что хотя каждая макромолекула генерируется случайно, распределение большого числа макромолекул по размерам (расстоянию между концами цепи) подчиняется определенному закону. Больше о нём вы можете узнать в учебнике "Высокомолекулярные соединения" под ред. А. Б. Зезина на с. 39.

\begin{equation}
\overline{R^2} = Nb^2\left[\frac{1+\cos\theta}{1-\cos\theta} - \frac{2}{N}\cos\theta\frac{1-(\cos\theta)^N}{(1-\cos\theta)^2}\right]
\end{equation}

\begin{equation}
\overline{R^2} \cong Nb^2\frac{1+\cos\theta}{1-\cos\theta}
\end{equation}

In [ ]:
calculated_mean_distance = mean([x**2 for x in distances])

a = (1 + cos(theta)) / (1 - cos(theta))
theoretical_mean_distance = SEGMENT_COUNT * (LENGTH ** 2) * a

b = 2 / SEGMENT_COUNT * cos(theta) * (1 - (cos(theta) ** SEGMENT_COUNT)) / (1 - cos(theta)) ** 2
precise_theoretical_mean_distance = SEGMENT_COUNT * (LENGTH ** 2) * (a - b)

print("Получено:      ", math.sqrt(calculated_mean_distance))
print("Теор. (≈, N→∞):", math.sqrt(theoretical_mean_distance))
print("Теор. (точно): ", math.sqrt(precise_theoretical_mean_distance))

Теперь вы можете изменить длину (степень полимеризации) макромолекулы или валентный угол и посмотреть, как при этом будет изменяться вид и расстояние между концами макромолекулы. Для этого измените значения переменных `SEGMENT_COUNT` (число мономерных звеньев / сегментов) и `ANGLE_DEG` (валентный угол) в начале кода первого шага и заново выполните первый шаг и последующий.


*© А. А. Коригодский*